<img src ="images/ch4_0.png">

After the three practical examples in chapter 3, you should be starting to feel familiar with how to approach classification and regression problems using neural networks, and you’ve witnessed the central problem of machine learning: overfitting. This chapter will formalize some of your new intuition into a solid conceptual framework for attacking and solving deep-learning problems. We’ll consolidate all of these concepts—**model evaluation, data preprocessing and feature engineering, and tackling overfitting**—into a detailed seven-step workflow for tackling any machine-learning task.

# 4.1. Four branches of machine learning

Machine-learning algorithms generally fall into four broad categories:
- Supervised Learning
- Unsupervised Learning
- Self-supervised learning
- Reinforcement learning

## 4.1.1. Supervised learning
This is by far the most common case. It consists of learning to map input data to known targets (also called annotations), given a set of examples (often annotated by humans). All four examples you’ve encountered in this book so far were canonical examples of supervised learning. Generally, almost all applications of deep learning that are in the spotlight these days belong in this category, such as **optical character recognition, speech recognition, image classification, and language translation**

<img src ="images/ch4_0_1.png">

## 4.1.2. Unsupervised learning

- This branch of machine learning consists of finding interesting transformations of the input data **without the help of any targets**, for the purposes of **data visualization, data compression, or data denoising, or to better understand the correlations present in the data at hand**. 
- Unsupervised learning is the bread and butter of data analytics, and it’s often a necessary step in better understanding a dataset before attempting to solve a supervised-learning problem. 
- **Dimensionality reduction** and **clustering** are well-known categories of unsupervised learning.

## 4.1.3. Self-supervised learning

- This is a specific instance of supervised learning, but it’s different enough that it deserves its own category. 
- Self-supervised learning is **supervised learning without human-annotated labels**—you can think of it as supervised learning without any humans in the loop. 
- There are still labels involved (because the learning has to be supervised by something), but they’re generated from the input data, typically using a heuristic algorithm.
- For instance, autoencoders are a well-known instance of self-supervised learning, where the generated targets are the input, unmodified. 
- In the same way, **trying to predict the next frame in a video, given past frames, or the next word in a text, given previous words, are instances of self-supervised learning (temporally supervised learning, in this case: supervision comes from future input data)**. 
- Note that the distinction between supervised, self-supervised, and unsupervised learning can be blurry sometimes—these categories are more of a continuum without solid borders. 
- Self-supervised learning can be reinterpreted as either supervised or unsupervised learning, depending on whether you pay attention to the learning mechanism or to the context of its application (예를 들면, 오토인코더는 타깃이 있고 손실 함수를 최소화하도록 학습되기 때문에 지도 학습으로 보이지만, 입력 데이터의 차원 축소 용도로 사용될 때는 비지도 학습으로 볼 수 있습니다.)

## 4.1.4. Reinforcement learning

- Recently started to get a lot of attention after Google DeepMind successfully applied it to learning to play Atari games (and, later, learning to play Go at the highest level). 
- In reinforcement learning, an agent receives information about its environment and learns to choose actions that will maximize some reward. 
- For instance, a neural network that “looks” at a video-game screen and outputs game actions in order to maximize its score can be trained via reinforcement learning.

Currently, reinforcement learning is mostly a research area and hasn’t yet had significant practical successes beyond games. In time, however, we expect to see reinforcement learning take over an increasingly large range of real-world applications: **self-driving cars, robotics, resource management, education, and so on.** It’s an idea whose time has come, or will come soon.

<img src ="images/ch4_0_2.png">

<img src ="images/ch4_0_3.png">

## 4.2. Evaluating machine-learning models

- In the three examples presented in chapter 3, we split the data into a training set, a validation set, and a test set. 
- The reason not to evaluate the models on the same data they were trained on quickly became evident: after just a few epochs, all three models began to overfit. 
- That is, their performance on never-before-seen data started stalling (or worsening) compared to their performance on the training data—which always improves as training progresses.


In machine learning, the goal is to achieve models that generalize—that perform well on never-before-seen data—and overfitting is the central obstacle. You can only control that which you can observe, so it’s crucial to be able to reliably measure the generalization power of your model. The following sections look at strategies for mitigating overfitting and maximizing generalization. In this section, we’ll focus on how to measure generalization: how to evaluate machine-learning models.

### 4.2.1. Training, validation, and test sets

Evaluating a model always boils down to splitting the available data into three sets: **training, validation, and test**. 

**You train on the training data and evaluate your model on the validation data. Once your model is ready for prime time, you test it one final time on the test data.**

You may ask, why not have two sets: a training set and a test set? You’d train on the training data and evaluate on the test data. Much simpler!

The reason is that developing a model always involves tuning its configuration: for example, choosing the number of layers or the size of the layers (called the hyper-parameters of the model, to distinguish them from the parameters, which are the network’s weights). You do this tuning by using as a feedback signal the performance of the model on the validation data. In essence, this tuning is a form of learning: a search for a good configuration in some parameter space. As a result, tuning the configuration of the model based on its performance on the validation set can quickly result in overfitting to the validation set, even though your model is never directly trained on it.

Central to this phenomenon is the notion of **information leaks (정보 누설)**. Every time you tune a hyperparameter of your model based on the model’s performance on the validation set, some information about the validation data leaks into the model. If you do this only once, for one parameter, then very few bits of information will leak, and your validation set will remain reliable to evaluate the model. But if you repeat this many times—running one experiment, evaluating on the validation set, and modifying your model as a result—then you’ll leak an increasingly significant amount of information about the validation set into the model.

At the end of the day, you’ll end up with a model that performs artificially well on the validation data, because that’s what you optimized it for. You care about performance on completely new data, not the validation data, so you need to use a completely different, never-before-seen dataset to evaluate the model: the test dataset. Your model shouldn’t have had access to any information about the test set, even indirectly. If anything about the model has been tuned based on test set performance, then your measure of generalization will be flawed.

Splitting your data into training, validation, and test sets may seem straightforward, but there are a few advanced ways to do it that can come in handy **when little data is available**. 

Let’s review three classic evaluation recipes: **simple hold-out validation, K-fold validation, and iterated K-fold validation with shuffling.

#### (1). Simple hold-out validation

Set apart some fraction of your data as your test set. Train on the remaining data, and evaluate on the test set. As you saw in the previous sections, in order to prevent information leaks, you shouldn’t tune your model based on the test set, and therefore you should also reserve a validation set.

Schematically, hold-out validation looks like figure 4.1. The following listing shows a simple implementation.

(그림 4-1 and 4-2 와 코드 4-1 and 4-2는 테스트 세트를 이미 떼어 놓은 후를 가정한 것임. 여기처럼 직접 데이터를 나누기보다는 종종 사이킷런의 train_test_split() 함수를 상용하여 훈련, 검증, 테스트 세트로 나는 것이 편리함)


- validation data를 train 데이터에 다시 넣고 학습시킴.

<img src ="images/ch4_1.png">

<img src ="images/코드4_1.png">
<img src ="images/코드4_1_1.png">

In [ ]:
num_validation_samples = 10000

np.random.shuffle(data)

validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

training_data = data[:]

model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

# At this point you can tune your model,
# retrain it, evaluate it, tune it again...

model = get_model()
model.train(np.concatenate([training_data,
                            validation_data]))
test_score = model.evaluate(test_data)

This is the simplest evaluation protocol, and it suffers from one flaw: if little data is available, then your validation and test sets may contain too few samples to be statistically representative of the data at hand. 

**This is easy to recognize: if different random shuffling rounds of the data before splitting end up yielding very different measures of model performance, then you’re having this issue.** 

K-fold validation and iterated K-fold validation are two ways to address this, as discussed next.

#### (1). K-fold validation

With this approach, you split your data into K partitions of equal size. 

For each partition i, train a model on the remaining K – 1 partitions, and evaluate it on partition i. 

Your final score is then the averages of the K scores obtained. 

This method is helpful when the performance of your model shows significant variance based on your train-test split. 

Like hold-out validation, this method doesn’t exempt you from using a distinct validation set for model calibration.

Schematically, K-fold cross-validation looks like figure 4.2. Listing 4.2 shows a simple implementation.

<img src ="images/Fig4-2.png">

K-fold cross-validation은 사이킷런의 cross_validate() 함수를 사용하여 쉽게 구현할 수 있습니다. 이 함수를 사용하려면 케라스 모델을 사이킷런과 호환되도록 KerasClassifier나 KerasRegressor 클래스로 모델을 감싸야 함.

<img src ="images/코드4_2.png">
<img src ="images/코드4_2_1.png">

In [ ]:
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:
     num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] +
     data[num_validation_samples * (fold + 1):]
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)

validation_score = np.average(validation_scores)

model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

#### (3)Iterated K-fold validation with shuffling

This one is for situations in which you have relatively little data available and you need to evaluate your model as precisely as possible. I’ve found it to be extremely helpful in Kaggle competitions. 

It consists of applying K-fold validation multiple times, shuffling the data every time before splitting it K ways. 

The final score is the average of the scores obtained at each run of K-fold validation. 

Note that you end up training and evaluating P × K models (where P is the number of iterations you use), which can be very expensive.

<img src ="images/역주4_6.png">

[참고예제](https://tensorflow.blog/2017/12/27/%EB%B0%98%EB%B3%B5-%EA%B5%90%EC%B0%A8-%EA%B2%80%EC%A6%9D/)

### 4.2.2. Things to keep in mind

Keep an eye out for the following when you’re choosing an evaluation protocol:

- **Data representativeness (대표성 있는 데이터)**— You want both your training set and test set to be representative of the data at hand. For instance, if you’re trying to classify images of digits, and you’re starting from an array of samples where the samples are ordered by their class, taking the first 80% of the array as your training set and the remaining 20% as your test set will result in your training set containing only classes 0–7, whereas your test set contains only classes 8–9. This seems like a ridiculous mistake, but it’s surprisingly common. For this reason, **you usually should randomly shuffle your data before splitting it into training and test sets**.
- **The arrow of time (시간의 방향)**— **If you’re trying to predict the future given the past (for example, tomorrow’s weather, stock movements, and so on), you should not randomly shuffle your data before splitting it, because doing so will create a temporal leak:** your model will effectively be trained on data from the future. In such situations, you should always make sure all data in your test set is posterior to the data in the training set(시간순서대로 train, val, test를 나눔).

-> 시간 정보가 있는 것은 기간별로 나누어주어야 함. 막 shuffle 하면 X

- **Redundancy in your data**— If some data points in your data appear twice (fairly common with real-world data), then shuffling the data and splitting it into a training set and a validation set will result in redundancy between the training and validation sets. In effect, you’ll be testing on part of your training data, which is the worst thing you can do! **Make sure your training set and validation set are disjoint (train data와 validation data에 중복 examples이 없는 지 확인해야함)**.